# Grafy - `NetworkX` - szybki przegląd

## Konfiguracja środowiska (VS Code + Python 3.12+)

:::{seealso} [Dokumentacja - instalacja NetworkX](https://networkx.org/documentation/stable/install.html)
:::

Najlepsze rozwiązanie dla Win11: instalacja WSL Ubuntu 24.04 + instalacja `networkx[default,extra]`

0. Python3.12
0. VSCode + plugin: Jupyter (od Microsoft)
1. Definiowanie środowiska wirtualnego `venv`
2. Instalacja bibliotek networkx: `pip install networkx[default]`:
  - instaluje `networkx` oraz biblioteki dodatkowe, wymagane: `numpy`, `scipy`, `pandas`, `matplotlib`, ... .
  - `extra` --> instaluje dodatkowe biblioteki (`PyGraphViz`, `pydoot`, `lxml`). Wymagana instalacja `graphviz` lokalnie ([instalacja PyGraphViz](https://pygraphviz.github.io/documentation/stable/install.html))


In [ ]:
# importowanie biblioteki networkx
import networkx as nx
display(nx.__version__)

In [ ]:
# biblioteki umożliwiające prostą wizualizację grafów
import matplotlib.pyplot as plt

:::{seealso} Oficjalny [tutorial NetworkX](https://networkx.org/documentation/stable/tutorial.html) :::

## Definiowanie grafu

In [ ]:
# klasa `Graph` - graf nieskierowany, prosty
G = nx.Graph()

# dodanie pojedynczego węzła o podanej etykiecie
G.add_node('a') 

# dodanie wielu węzłów z obiektu iterowalnego (np. listy)
G.add_nodes_from(['b', 'c'])

# dodanie pojedynczej krawędzi, graf nieskierowany, kolejność nie ma znaczenia
G.add_edge('a', 'b')

# dodanie wielu krawędzi, uzupełnienie brakujących węzłów
edges_to_add = [('a', 'c'), ('b', 'c'), ('c', 'd')]
G.add_edges_from(edges_to_add)

# rysowanie grafu
nx.draw(G, 
        with_labels=True,
        #node_color='blue',
        #node_size=1600,
        #font_color='white',
        #font_size=16,
        )

In [ ]:
# parametry grafu
display( G.nodes ) # `nodes` jest property, zwraca iterowalny obiekt `NodeView`
print( f"liczba węzłów: {G.number_of_nodes()}")
print( G.edges )
print( f"liczba krawędzi: {G.number_of_edges()}")

In [ ]:
# sąsiedzi węzła
print(G.neighbors('b')) # `neighbors`` zwraca iterator
for v in G.neighbors('b'):
    print(v, end =" ")
print()
print( list(G.neighbors('b')))

`NetworkX` pozwala na dostęp do cech grafu na dwa sposoby:

* jako metody obiektu `Graph`: `G.<method_name>(<arguments>)` lub `G.<property>` - głównie strukturalne cechy grafu
* jako funkcje pakietu/modułu: `nx.<function_name>(G, <arguments>)` - głównie algorytmy grafowe

In [ ]:
print( G.degree('b') )
print( nx.degree(G, 'b') )

# print( G.is_tree() ) # nie ma takiej metody
print( nx.is_tree(G) )      # czy jest drzewem
print( nx.is_connected(G) ) # czy jest spójny

In [ ]:
# przynależność węzła/krawędzi do grafu

print( 'b' in G.nodes )
print( G.has_node('b') )
print( G.has_edge('b', 'a'))
print( ('a', 'b') in G.edges )

### Ćwiczenie 1

Uzupełnij kod funkcji `liscie(G: Graph) -> Iterable` zwracającej sekwencję węzłów grafu nieskierowanego prostego, które są liśćmi (mają tylko jednego sąsiada).

In [ ]:
# Ćwiczenie 1
from typing import Iterable
from networkx import Graph # type: ignore

def liscie(G: Graph) -> Iterable:
    # raise NotImplementedError("brak kodu")

In [ ]:
# test - ćwiczenie 1
G_test = nx.Graph()
G_test.add_edges_from([
        ('a', 'b'),
        ('a', 'd'),
        ('c', 'd'),
    ])

assert set(liscie(G_test)) == {'c', 'b'}
print(sorted(list(liscie(G_test))))

## Reprezentacje grafu

In [ ]:
# Eksport grafu do macierzy sąsiedztwa

G1 = nx.Graph()
G1.add_edges_from([(1, 2), (2, 3), (3, 1)])

# Macierz sąsiedztwa jako tablica numpy
adj_matrix = nx.to_numpy_array(G1)
print(adj_matrix, end="\n\n")

# Macierz sąsiedztwa jako dataframe pandas
import pandas as pd
adj_matrix = nx.to_pandas_adjacency(G1)
print(adj_matrix, end="\n\n")


In [ ]:
# Eksport grafu do listy sąsiadów

# Iteracja po wierzchołkach z wypisaniem sąsiadów
for node in G.nodes:
    print(f"{node}: {list(G.neighbors(node))}")
print()

# G.adj - wewnętrzna reprezentacja grafu jako property typu AdjacencyView
print( G.adj )
print( type(G.adj) )

# nx.adjacency_data(G) - słownik, przechowujący strukturę grafu
print( nx.adjacency_data(G) )

### Wczytywanie grafu z pliku

1. Lista krawędzi, każda linia zawiera parę etykiet wierzchołków, np. dla pliku `graph.txt`
    ```txt
    1 2
    1 3
    2 3
    2 4
    ```

    polecenie:

    ```python
    G = nx.read_edgelist("graph.txt", nodetype=int)
    ```

2. Macierz sąsiedztwa, każda linia reprezentuje wiersz macierzy, za pomocą `numpy`:

    ```python
    # Wczytanie macierzy z pliku "matrix.txt"
    matrix = np.loadtxt("matrix.txt", dtype=int)

    # Utworzenie grafu z macierzy
    G = nx.from_numpy_matrix(matrix)
    ```

3. Lista sąsiedztwa, każda linia reprezentuje węzeł i jego sąsiadów:
    ```python
    # Wczytanie listy sąsiedztwa z pliku "adj_list.txt"
    adj_list = {}
    with open("adj_list.txt", "r") as f:
        for line in f:
            node, neighbors = line.strip().split(":")
            adj_list[int(node)] = [int(n) for n in neighbors.split()]

    # Utworzenie grafu z listy sąsiedztwa
    G = nx.from_dict_of_lists(adj_list)
    ```

4. Format GML ([_Graph Modeling Language_](https://en.wikipedia.org/wiki/Graph_Modelling_Language))
    ```python
    G = nx.read_gml("graph.gml")
    ```

5. Inne: `read_weighted_edgelist`, `read_adjlist`, ... --> dokumentacja

### Zapisywanie grafu do pliku tekstowego

* `write_edgelist`, `write_adjlist`, `write_gml`, ... -> dokumentacja

### Ćwiczenie 2

Wykonaj eksperymenty z wczytywaniem i zapisywaniem grafów do plików tekstowych, w różnych formatach

## Graf skierowany (_digraph_)

In [ ]:
D = nx.DiGraph() # graf skierowany, nieważony
D.add_edges_from([(1,2),(2,3),(3,2),(3,4),(3,5),(4,5),(4,6),(5,6),(6,4),(4,2)])
nx.draw(D, with_labels=True)

In [ ]:
print('Następniki dla 2:', list(D.successors(2)), "liczba: ", D.out_degree(2))
print('Poprzedniki dla 2:', list(D.predecessors(2)), "liczba: ", D.in_degree(2))
print( "Wierzchołki i ich stopnie: ", D.degree )
print("Sąsiedzi dla 2: ", list(D.neighbors(2)))

## Graf ważony

In [ ]:
H = nx.Graph() # graf nieskierowany, ważony
H.add_edge('A', 'B', weight=4)
H.add_weighted_edges_from([('B', 'C', 2), ('C', 'E', 3), ('D', 'E', 4), ('E', 'A', 5)])

layout = nx.spring_layout(H)
m = nx.draw_networkx(H, pos=layout, with_labels=True)
m = nx.draw_networkx_edge_labels(H, pos=layout, edge_labels=nx.get_edge_attributes(H, 'weight'))

In [ ]:
D = nx.DiGraph()
D.add_edge("A", "B", weight=2)
D.add_edge("B", "A", weight=3)
D.add_edge("A", "C", weight=4)
D.add_edge("C", "A", weight=5)
D.add_edge("A", "D", weight=6)
D.add_edge("B", "D", weight=7)
D.add_edge("C", "D", weight=8)

positions = {"A": (0, 0), "B": (1, -2), "C": (1, 2), "D": (2, 0)}
nx.draw_networkx_nodes(D, pos=positions, node_size=500)
nx.draw_networkx_labels(D, pos=positions, font_color="w")
nx.draw_networkx_edges(
    D,
    pos=positions,
    arrowstyle="->",
    connectionstyle="arc3,rad=0.3",
)
nx.draw_networkx_edge_labels(D, pos=positions, edge_labels=nx.get_edge_attributes(D, 'weight'), label_pos=0.2);

In [ ]:
#!pip install pydot
dot = nx.nx_pydot.to_pydot(D)
print(dot)

## Generatory grafów

In [ ]:
g = nx.barabasi_albert_graph(10, 2)
nx.draw(g, with_labels=True)

In [ ]:
g = nx.complete_graph(5)
nx.draw(g, pos = nx.shell_layout(g), with_labels=False)

In [ ]:
g = nx.erdos_renyi_graph(20,0.15)
nx.draw(g, with_labels=True)

## Ścieżki, cykle, spójność

In [ ]:
nx.draw(G, with_labels=True)

In [ ]:
# ścieżki
print("Czy jest ścieżka od 'a' do 'd':", nx.has_path(G, 'a', 'd') )

print("Ścieżki bez cykli:", list(nx.all_simple_paths(G, 'a', 'd')))
print("Najkrótsza ścieżka:", nx.shortest_path(G, 'a', 'd'), "ma długość:",  nx.shortest_path_length(G, 'a', 'd'))

In [ ]:
# spójność
nx.is_connected(G)

In [ ]:
G2 = nx.Graph()

nx.add_cycle(G2, (1,2,3))
G2.add_edge(4,5)

nx.draw(G2, with_labels=True)

In [ ]:
print( nx.is_connected(G2) )
print( nx.has_path(G2, source=3, target=5))
print("Liczba składowych spójnych:", nx.number_connected_components(G2))
print("Lista składowych spójnych:", list(nx.connected_components(G2)))
print("Maksymalna składowa spójna:", max(nx.connected_components(G2), key=len))

# podgraf zbudowany na węzłach maksymalnej składowej spójnej
core_nodes = max(nx.connected_components(G2), key=len)
core = G2.subgraph( core_nodes )
nx.draw(core, with_labels=True)

### Ścieżki i spójność dla grafów skierowanych

In [ ]:
D1 = nx.DiGraph()
D1.add_edges_from([
    (1,2),
    (2,3),
    (3,2), (3,4), (3,5),
    (4,2), (4,5), (4,6),
    (5,6),
    (6,4),
])
nx.draw(D1, with_labels=True)

In [ ]:
print(nx.has_path(D1, 1, 4))
print(nx.has_path(D1, 4, 1))

In [ ]:
print(nx.shortest_path(D1, 2, 5))
print(nx.shortest_path(D1, 5, 2))

W grafach skierowanych występują dwa rodzaje spójności:

* składowa silnie spójna (ang. _strongly connected_) - z dowolnego wierzchołka możemy dostać się do dowolnego innego wierzchołka, zachowując kierunkowość krawędzi

* składowa słabo spójna (ang. _weakly connected_) - ​​istnieje ścieżka pomiędzy każdą parą węzłów, niezależnie od kierunku

In [ ]:
print(nx.is_strongly_connected(D1))
print(list(nx.strongly_connected_components(D1)))

print(nx.is_weakly_connected(D1))
# print(nx.is_connected(D1)) # nie wolno użyć dla grafów skierowanych
print(list(nx.weakly_connected_components(D1)))

## Przechodzenie po grafie

* [NetworkX. Traversal. Depth First Search](https://networkx.org/documentation/stable/reference/algorithms/traversal.html#module-networkx.algorithms.traversal.depth_first_search)

* [NetworkX. Traversal. Breadth First Search](https://networkx.org/documentation/stable/reference/algorithms/traversal.html#module-networkx.algorithms.traversal.breadth_first_search)

## Ćwiczenie 3

Ćwiczenie dotyczy eksploracji grafu połączeń między lotniskami w USA. Wierzchołki grafu etykietowane są kodami [IATA](https://en.wikipedia.org/wiki/List_of_airports_by_IATA_airport_code:_A). W pliku w formacie [GraphML](https://en.wikipedia.org/wiki/GraphML) zapisane są również bezpośrednie połączenia między lotniskami. Zakładamy, że graf jest **nieskierowany**. Dane pochodzą sprzed ok. 8 lat i nie obejmują wszystkich połączeń (są to dane do ćwiczenia).

UWAGA: rozwiązując zadania możesz - na podstawie definicji grafu - opracowywać algorytmy i wykonywać obliczenia. Możesz również eksplorować dokumentację projektu `NetworkX` i odnaleźć dedykowane funkcje.

Postawione pytania mają charakter otwarty. Sposób podejścia do udzielenia odpowiedzi na te pytania zależy od Ciebie.

In [ ]:
import networkx as nx
import requests

# URL pliku z opisem grafu
url = "https://raw.githubusercontent.com/URK-KIPLiIS/Python-Data/main/airports/openflights_usa.graphml"

# Pobranie treści pliku z URL
response = requests.get(url)
response.raise_for_status()  # Sprawdzenie, czy pobranie zakończyło się sukcesem

# Wczytanie grafu z pobranej treści, która jest już stringiem
G = nx.parse_graphml(response.text)

# Wyświetlenie grafu
print(G.nodes)
print(G.edges)

G.nodes['IND']

In [ ]:
display( G.number_of_nodes() )
G.number_of_edges()
print( list( nx.all_shortest_paths(G, 'GLH', 'AUK') ))

### Zadanie 1

Czy jest bezpośredni lot na trasie Indianapolis–Fairbanks na Alasce (FAI)? Lot bezpośredni to taki, w którym nie ma przesiadek.

Gdybym chciał polecieć z Indianapolis do Fairbanks na Alasce, jaki byłby plan obejmujący najmniejszą liczbę przesiadek?

### Zadanie 2

Wyobraź sobie, że chcesz zaplanować podróż samolotem z lotniska w Nowym Jorku (JFK) do lotniska w Los Angeles (LAX). Chcesz znaleźć trasę, która obejmuje co najwyżej 3 przesiadki. Wypisz wszystkie takie trasy.
> Wykorzystaj algorytm BFS do znalezienia tych tras?

### Zadanie 3

Czy można podróżować z dowolnego lotniska w USA na inne lotnisko w USA, ewentualnie korzystając z lotów przesiadkowych? Innymi słowy, czy istnieje ścieżka w sieci pomiędzy każdą możliwą parą lotnisk?


### Zadanie 4 - struktura grafu

* Jaki jest stopień węzła reprezentującego lotnisko w Chicago (ORD)?
* Ile jest węzłów w tym grafie?
* Ile jest krawędzi w tym grafie?
* Jaki jest średni stopień węzła w tym grafie?
* Jaki jest średni dystans między dwoma losowo wybranymi węzłami w tym grafie?
    > Średni dystans ścieżkowy jest ważną miarą w analizie sieci, ponieważ pozwala na ocenę "rozmiaru" sieci i łatwości poruszania się po niej. Im mniejszy średni dystans ścieżkowy, tym bardziej "gęsta" jest sieć i tym łatwiej jest dotrzeć z jednego węzła do drugiego. `nx.average_shortest_path_length` działa tylko dla grafów spójnych. Obliczenie średniego dystansu ścieżkowego może być czasochłonne dla dużych grafów.

### Zadanie 5 - połączenia

* Które lotniska są bezpośrednio połączone z lotniskiem w Los Angeles (LAX)?
* Czy istnieje ścieżka między lotniskiem w Nowym Jorku (JFK) a lotniskiem w San Francisco (SFO)?
* Jaka jest najkrótsza ścieżka między lotniskiem w Miami (MIA) a lotniskiem w Seattle (SEA)?
* Ile jest różnych ścieżek między lotniskiem w Chicago (ORD) a lotniskiem w Denver (DEN)?
* Czy graf połączeń jest spójny, a jeśli nie, ile liczy składowych spójnych?
* Czy istnieją mosty w tym grafie? Jeśli są, to między którymi lotniskami.
* Czy istnieją punkty artykulacji w tym grafie? (Czy usunięcie jakiegokolwiek węzła spowodowałoby rozłączenie grafu?)

**Podpowiedzi:** `nx.bridges`, `nx.articulation_points`, [Most (teoria grafów) @Wikipedia](https://pl.wikipedia.org/wiki/Most_(teoria_graf%C3%B3w)), [Punkt artykulacji @Wikipedia](https://pl.wikipedia.org/wiki/Punkt_artykulacji)